# Path

In [1]:
import sys
print(sys.argv[0])
import os
print (os.getcwd())#获得当前工作目录
print (os.path.abspath('.'))#获得当前工作目录
print (os.path.abspath('..'))#获得当前工作目录的父目录
print (os.path.abspath(os.curdir))#获得当前工作目录
# print(os.path.abspath('..')+'/src/sample_code_submission')
# os.chdir(os.path.abspath('..')+'/src/sample_code_submission')
# os.path.abspath('..')+'/src/sample_code_submission'
from pprint import pprint

/Users/xijunli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py
/Users/xijunli/Desktop/KDDCup2019/jupyter
/Users/xijunli/Desktop/KDDCup2019/jupyter
/Users/xijunli/Desktop/KDDCup2019
/Users/xijunli/Desktop/KDDCup2019/jupyter


# Data Input

In [2]:
# pylint: disable=wrong-import-order, wrong-import-position, import-error
# pylint: disable=missing-docstring
import base64
from datetime import datetime
import os
from os.path import join
import sys

#  os.system("pip3 install cryptography")


def mprint(msg):
    """info"""
    cur_time = datetime.now().strftime('%m-%d %H:%M:%S')
    print(f"INFO  [{cur_time}] {msg}")


mprint("Import Model")

import json
import signal
import time
from contextlib import contextmanager
import numpy as np
import pandas as pd
import math

TYPE_MAP = {
    'time': str,
    'cat': str,
    'multi-cat': str,
    'num': np.float64
}


class TimeoutException(Exception):
    pass


class Timer:
    def __init__(self):
        self.duration = 0
        self.total = None
        self.remain = None
        self.exec = None

    def set(self, time_budget):
        self.total = time_budget
        self.remain = time_budget
        self.exec = 0

    @contextmanager
    def time_limit(self, pname):
        def signal_handler(signum, frame):
            raise TimeoutException("Timed out!")
        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(self.remain)
        start_time = time.time()
        try:
            yield
        finally:
            exec_time = time.time() - start_time
            signal.alarm(0)
            self.exec += exec_time
            self.duration += exec_time
            remain_time = math.ceil(self.total - self.exec)
            self.remain = remain_time

            mprint(f'{pname} success, time spent so far {self.exec} sec')


def read_train(datapath, info):
    train_data = {}
    for table_name, columns in info['tables'].items():
        mprint(f'Table name: {table_name}')

        table_dtype = {key: TYPE_MAP[val] for key, val in columns.items()}

        if table_name == 'main':
            table_path = join(datapath, 'train', 'main_train.data')
        else:
            table_path = join(datapath, 'train', f'{table_name}.data')

        date_list = [key for key, val in columns.items() if val == 'time']

        train_data[table_name] = pd.read_csv(
            table_path, sep='\t', dtype=table_dtype, parse_dates=date_list,
            date_parser=lambda millisecs: millisecs if np.isnan(
                float(millisecs)) else datetime.fromtimestamp(
                    float(millisecs)/1000))

    # get train label
    train_label = pd.read_csv(
        join(datapath, 'train', 'main_train.solution'))['label']
    return train_data, train_label


def read_info(datapath):
    mprint('Read info')
    with open(join(datapath, 'train', 'info.json'), 'r') as info_fp:
        info = json.load(info_fp)
    mprint(f'Time budget for this task is {info["time_budget"]} sec')
    return info


def read_test(datapath, info):
    # get test data
    main_columns = info['tables']['main']
    table_dtype = {key: TYPE_MAP[val] for key, val in main_columns.items()}

    table_path = join(datapath, 'test', 'main_test.data')

    date_list = [key for key, val in main_columns.items() if val == 'time']

    test_data = pd.read_csv(
        table_path, sep='\t', dtype=table_dtype, parse_dates=date_list,
        date_parser=lambda millisecs: millisecs if np.isnan(
            float(millisecs)) else datetime.fromtimestamp(
                float(millisecs) / 1000))
    return test_data


def write_predict(output_dir, dataname, prediction):
    os.makedirs(output_dir, exist_ok=True)
    prediction.rename('label', inplace=True)
    prediction.to_csv(
        join(output_dir, f'{dataname}.predict'), index=False, header=True)


INFO  [04-22 11:05:36] Import Model


In [3]:
ROOT_DIR = os.path.abspath('/Users/xijunli/Desktop/KDDCup2019/starting_kit_0401/')
DIRS = {
        'input': join(ROOT_DIR, 'sample_data'),
        'output': join(ROOT_DIR, 'sample_predictions'),
        'program': join(ROOT_DIR, 'ingestion_program'),
        'submission': join(ROOT_DIR, 'sample_code_submission')
}

datanames = sorted(os.listdir(DIRS['input']))
mprint(f'Datanames: {datanames}')
timer = Timer()
dataname = datanames[0]
predictions = {}
mprint(f'Read data: {dataname}')
datapath = join(DIRS['input'], dataname)
info = read_info(datapath)
timer.set(info['time_budget'])
train_data, train_label = read_train(datapath, info)

INFO  [04-22 11:05:37] Datanames: ['K', 'L']
INFO  [04-22 11:05:37] Read data: K
INFO  [04-22 11:05:37] Read info
INFO  [04-22 11:05:37] Time budget for this task is 6000 sec
INFO  [04-22 11:05:37] Table name: main
INFO  [04-22 11:05:37] Table name: table_1
INFO  [04-22 11:05:37] Table name: table_2
INFO  [04-22 11:05:37] Table name: table_3


# AutoML Model

## Construct Model

In [4]:
os.chdir(os.path.abspath('..')+'/src/sample_code_submission/')
print(os.path.abspath('.'))

import copy
import numpy as np
import pandas as pd

from automl import predict, train, validate
from CONSTANT import MAIN_TABLE_NAME
from merge import merge_table
from preprocess import clean_df, clean_tables, feature_engineer
from util import Config, log, show_dataframe, timeit

config = Config(info)
tables = copy.deepcopy(train_data)

/Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission


/Users/xijunli/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


## fit

### clean_tables

In [5]:
import os
import time
from collections import defaultdict, deque

import numpy as np
import pandas as pd

import CONSTANT
from util import Config, Timer, log, timeit

NUM_OP = [np.std, np.mean]

for tname in tables:
    log(f"cleaning table {tname}")
    clean_df(tables[tname])

cleaning table main

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.01 sec.
End   [clean_df]. Time elapsed: 0.01 sec.
cleaning table table_1

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.03 sec.
End   [clean_df]. Time elapsed: 0.03 sec.
cleaning table table_2

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.01 sec.
End   [clean_df]. Time elapsed: 0.01 sec.
cleaning table table_3

Start [clean_df]:
----Start [fillna]:
----End   [fillna]. Time elapsed: 0.00 sec.
End   [clean_df]. Time elapsed: 0.01 sec.


### X = merge_table(Xs, self.config)

In [6]:
from collections import defaultdict, deque

import numpy as np
import pandas as pd

import CONSTANT
from util import Config, Timer, log, timeit
from merge import bfs, dfs
# def merge_table(tables, config):

graph = defaultdict(list)
for rel in config['relations']:
    ta = rel['table_A']
    tb = rel['table_B']
    graph[ta].append({
            "to": tb,
            "key": rel['key'],
            "type": rel['type']
    })
    graph[tb].append({
            "to": ta,
            "key": rel['key'],
            "type": '_'.join(rel['type'].split('_')[::-1])
    })
bfs(CONSTANT.MAIN_TABLE_NAME, graph, config['tables'])
X = dfs(CONSTANT.MAIN_TABLE_NAME, config, tables, graph)

enter main
enter table_1
leave table_1
join main <--many_to_one--nt table_1

Start [join]:
End   [join]. Time elapsed: 0.03 sec.
enter table_2
leave table_2
join main <--many_to_one--nt table_2

Start [join]:
End   [join]. Time elapsed: 0.02 sec.
enter table_3
leave table_3
join main <--many_to_one--nt table_3

Start [join]:
End   [join]. Time elapsed: 0.01 sec.
leave main


#### bfs(CONSTANT.MAIN_TABLE_NAME, graph, config['tables'])

In [8]:
from merge import dfs
tconfig = config["tables"]
root_name = CONSTANT.MAIN_TABLE_NAME

tconfig[CONSTANT.MAIN_TABLE_NAME]['depth'] = 0
queue = deque([root_name])
while queue:
    u_name = queue.popleft()
    for edge in graph[u_name]:
        v_name = edge['to']
        if 'depth' not in tconfig[v_name]:
            tconfig[v_name]['depth'] = tconfig[u_name]['depth'] + 1
            queue.append(v_name)

print("graph")
pprint(graph)
print("config")
pprint(config)

dfs(CONSTANT.MAIN_TABLE_NAME, config, tables, graph)

graph
defaultdict(<class 'list'>,
            {'main': [{'key': ['c_01'], 'to': 'table_1', 'type': 'many_to_one'},
                      {'key': ['c_02'], 'to': 'table_2', 'type': 'many_to_one'},
                      {'key': ['c_02'],
                       'to': 'table_3',
                       'type': 'many_to_one'}],
             'table_1': [{'key': ['c_01'],
                          'to': 'main',
                          'type': 'one_to_many'}],
             'table_2': [{'key': ['c_02'],
                          'to': 'main',
                          'type': 'one_to_many'}],
             'table_3': [{'key': ['c_02'],
                          'to': 'main',
                          'type': 'one_to_many'}]})
config
{'start_time': 1550654179, 'time_budget': 6000, 'time_col': 't_01', 'tables': {'main': {'type': {'n_1': 'num', 'n_2': 'num', 'n_3': 'num', 't_01': 'time', 'c_02': 'cat', 'c_01': 'cat'}, 'depth': 0}, 'table_1': {'type': {'c_01': 'cat', 'c_1': 'cat', 'c_2': 'cat', 'n_

(Pdb) p v
                       c_01                   c_1                   c_2  \
0       7857015818808240760  14771401151246747869                     0   
1       1940685345704822677  14771401151246747869   9722599372422752642   
2         68653636337931097  14771401151246747869  12236148022553120681   
3       9702724967421813869  14771401151246747869                     0   
4      15552523380951821167     34453685010072459   6786087198600974901   
5      17710186355858817716  14771401151246747869                     0   
6      12982109506759899552     34453685010072459   8286680241381910257   
7      16587249206215947911  14771401151246747869  10211136965740185639   
8       8611478402124189380  14771401151246747869  10081650206047489431   
9       6685492506491068555  14771401151246747869  11740872891748772811   
10      4709959524028248941  14771401151246747869  12526130277778747046   
11      7331868530661020407  14771401151246747869   7638001807695707609   
12      5284232

(Pdb) p v_name
'table_1'
(Pdb) p v.columns
Index(['c_01', 'c_1', 'c_2', 'n_1', 'n_2', 'c_3', 'c_4', 'c_5', 'n_3', 'c_6',
       't_1', 'n_4', 'n_5', 'n_6', 'n_7', 'n_8', 'n_9', 'n_10', 'n_11', 'n_12',
       'n_13', 'n_14', 'n_15', 'n_16', 'n_17', 'n_18', 'n_19', 'n_20', 'n_21',
       'n_22', 'c_7', 'n_23', 'n_24', 'n_25', 'n_26', 'n_27'],
      dtype='object')
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(37)join()
-> v = v.set_index(key)
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(38)join()
-> v.columns = v.columns.map(lambda a: f"{a.split('_', 1)[0]}_{v_name}.{a}")
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(40)join()
-> return u.join(v, on=key)
(Pdb) p v.columns
Index(['c_table_1.c_1', 'c_table_1.c_2', 'n_table_1.n_1', 'n_table_1.n_2',
       'c_table_1.c_3', 'c_table_1.c_4', 'c_table_1.c_5', 'n_table_1.n_3',
       'c_table_1.c_6', 't_table_1.t_1', 'n_table_1.n_4', 'n_table_1

(Pdb) p u.index
RangeIndex(start=0, stop=9999, step=1)
(Pdb) p u.columns
Index(['n_1', 'n_2', 'n_3', 't_01', 'c_02', 'c_01', 'c_table_1.c_1',
       'c_table_1.c_2', 'n_table_1.n_1', 'n_table_1.n_2', 'c_table_1.c_3',
       'c_table_1.c_4', 'c_table_1.c_5', 'n_table_1.n_3', 'c_table_1.c_6',
       't_table_1.t_1', 'n_table_1.n_4', 'n_table_1.n_5', 'n_table_1.n_6',
       'n_table_1.n_7', 'n_table_1.n_8', 'n_table_1.n_9', 'n_table_1.n_10',
       'n_table_1.n_11', 'n_table_1.n_12', 'n_table_1.n_13', 'n_table_1.n_14',
       'n_table_1.n_15', 'n_table_1.n_16', 'n_table_1.n_17', 'n_table_1.n_18',
       'n_table_1.n_19', 'n_table_1.n_20', 'n_table_1.n_21', 'n_table_1.n_22',
       'c_table_1.c_7', 'n_table_1.n_23', 'n_table_1.n_24', 'n_table_1.n_25',
       'n_table_1.n_26', 'n_table_1.n_27'],
      dtype='object')
(Pdb) p u
        n_1  n_2  n_3                    t_01                  c_02  \
0       1.0  1.0  1.0 2017-05-05 00:00:04.518   3357165528367806273   
1       1.0  1.0  1.0 20

(Pdb) p u.Index
*** AttributeError: 'DataFrame' object has no attribute 'Index'
(Pdb) p u.indexes
*** AttributeError: 'DataFrame' object has no attribute 'indexes'
(Pdb) p u.index
RangeIndex(start=0, stop=9999, step=1)
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(91)dfs()
-> for edge in graph[u_name]:
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(92)dfs()
-> v_name = edge['to']
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(93)dfs()
-> if config['tables'][v_name]['depth'] <= config['tables'][u_name]['depth']:
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(96)dfs()
-> v = dfs(v_name, config, tables, graph)
(Pdb) n
enter table_2
leave table_2
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(97)dfs()
-> key = edge['key']
(Pdb) n
> /Users/xijunli/Desktop/KDDCup2019/src/sample_code_submission/merge.py(98)dfs()
-> type_ = edge['type']
(

(Pdb) n
> /Users/xijunli/anaconda3/lib/python3.6/site-packages/IPython/core/displayhook.py(263)__call__()
-> self.finish_displayhook()
(Pdb) n


,n_1,n_2,n_3,t_01,c_02,c_01,c_table_1.c_1,c_table_1.c_2,n_table_1.n_1,n_table_1.n_2,...,n_table_2.n_14,n_table_3.n_1,n_table_3.n_2,n_table_3.n_3,n_table_3.n_4,n_table_3.n_5,n_table_3.n_6,n_table_3.n_7,n_table_3.n_8,n_table_3.n_9
0,1.0,1.0,1.0,2017-05-05 00:00:04.518,3357165528367806273,14590537847198882611,14771401151246747869,0,1.000,1.0000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,2017-05-05 00:00:04.540,3357165528367806273,14590537847198882611,14771401151246747869,0,1.000,1.0000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,7.0,1.0,1.0,2017-05-05 00:00:04.584,6648472193321585296,14590537847198882611,14771401151246747869,0,1.000,1.0000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,8.5,1.0,1.0,2017-05-05 00:00:04.587,1801431999810370137,14590537847198882611,14771401151246747869,0,1.000,1.0000,...,1.0,64123.0,44.5,16912.0,10369.0,7658.5,29186.5,1214.5,1.0,1.0
4,7.0,1.0,1.0,2017-05-05 00:00:04.596,14475392532262432119,14590537847198882611,14771401151246747869,0,1.000,1.0000,...,1.0,1376.5,4.0,1376.5,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,2017-05-05 00:00:04.716,10189949036338060394,14590537847198882611,14771401151246747869,0,1.000,1.0000,...,1.0,121206.0,95.5,1753.0,32285.5,25477.0,61693.0,1651.0,1.0,1.0
6,19.0,1.0,1.0,2017-05-05 00:00:06.503,10469879042194243070,2502521354834814238,14771401151246747869,0,1.000,1.0000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,26.5,1.0,1.0,2017-05-05 00:00:06.532,2665815103940912083,2502521354834814238,14771401151246747869,0,1.000,1.0000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,2017-05-05 00:00:07.532,10503878146262666097,17939382046334772348,34453685010072459,14550647659428803805,177.013,36.9241,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,19.0,1.0,1.0,2017-05-05 00:00:07.541,10469879042194243070,17939382046334772348,34453685010072459,14550647659428803805,177.013,36.9241,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


--Return--
> /Users/xijunli/anaconda3/lib/python3.6/site-packages/IPython/core/displayhook.py(263)__call__()->None
-> self.finish_displayhook()
(Pdb) n
--Return--
> <ipython-input-8-d64ea348ccb2>(20)<module>()->None
-> dfs(CONSTANT.MAIN_TABLE_NAME, config, tables, graph)
(Pdb) n
> /Users/xijunli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py(2966)run_code()
-> sys.excepthook = old_excepthook
(Pdb) n
> /Users/xijunli/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py(2982)run_code()
-> outflag = False
(Pdb) q


BdbQuit: 

#### dfs(CONSTANT.MAIN_TABLE_NAME, config, tables, graph)

In [37]:
def dfs(u_name, config, tables, graph):
    pdb.set_trace()
    u = tables[u_name]
    log(f"enter {u_name}")
    for edge in graph[u_name]:
        v_name = edge['to']
        if config['tables'][v_name]['depth'] <= config['tables'][u_name]['depth']:
            continue

        v = dfs(v_name, config, tables, graph)
        key = edge['key']
        type_ = edge['type']

        if config['time_col'] not in u and config['time_col'] in v:
            continue

        if config['time_col'] in u and config['time_col'] in v:
            log(f"join {u_name} <--{type_}--t {v_name}")
            u = temporal_join(u, v, v_name, key, config['time_col'])
        else:
            log(f"join {u_name} <--{type_}--nt {v_name}")
            u = join(u, v, v_name, key, type_)

        del v

    log(f"leave {u_name}")
    return u

### Feature Engineer

In [12]:
clean_df(X)
X_fe = copy.deepcopy(X)
feature_engineer(X_fe,config)


----Start [clean_df]:
--------Start [fillna]:
--------End   [fillna]. Time elapsed: 0.03 sec.
----End   [clean_df]. Time elapsed: 0.03 sec.

----Start [feature_engineer]:
--------Start [transform_categorical_hash]:
--------End   [transform_categorical_hash]. Time elapsed: 0.12 sec.

--------Start [transform_datetime]:
--------End   [transform_datetime]. Time elapsed: 0.01 sec.
----End   [feature_engineer]. Time elapsed: 0.13 sec.


### Train

In [19]:
train(X_fe, train_label, config)


----Start [train]:
--------Start [train_lightgbm]:
------------Start [hyperopt_lightgbm]:
100%|██████████| 10/10 [00:01<00:00,  8.57it/s, best loss: -0.6957329351390847]
----------------auc = 0.6957 {'bagging_fraction': 0.8, 'bagging_freq': 22, 'feature_fraction': 0.5, 'learning_rate': 0.11643157545599682, 'max_depth': 6, 'min_child_weight': 9.195866326847561, 'num_leaves': 145, 'reg_alpha': 0.8629175589887947, 'reg_lambda': 1.935131592856683}
------------End   [hyperopt_lightgbm]. Time elapsed: 1.24 sec.
Training until validation scores don't improve for 30 rounds.
Early stopping, best iteration is:
[23]	valid_0's auc: 0.711347
--------End   [train_lightgbm]. Time elapsed: 1.40 sec.
----End   [train]. Time elapsed: 1.40 sec.
